In [1]:
import pandas as pd
from sklearn.utils import shuffle
df = pd.read_csv(f"EnglishOrSpanish/output_joint1.csv")
df = shuffle(df)
df

,English,Spanish
47504,"[2, 265, 1225, 334, 3724, 332, 300, 1675, 224,...","[2, 323, 2116, 393, 15255, 355, 559, 224, 3, 0..."
73273,"[2, 395, 1089, 411, 300, 1647, 1191, 224, 3, 0...","[2, 1193, 1247, 331, 1316, 1313, 224, 3, 0, 0,..."
6048,"[2, 426, 449, 538, 224, 3, 0, 0, 0, 0, 0, 0, 0...","[2, 833, 444, 584, 224, 3, 0, 0, 0, 0, 0, 0, 0..."
21959,"[2, 479, 1873, 2127, 328, 224, 3, 0, 0, 0, 0, ...","[2, 801, 1880, 328, 7743, 224, 3, 0, 0, 0, 0, ..."
95763,"[2, 270, 384, 332, 300, 3142, 659, 300, 2110, ...","[2, 535, 621, 322, 331, 3381, 780, 331, 1934, ..."
...,...,...
16908,"[2, 426, 449, 395, 1320, 224, 3, 0, 0, 0, 0, 0...","[2, 444, 1368, 1756, 224, 3, 0, 0, 0, 0, 0, 0,..."
89015,"[2, 341, 304, 473, 300, 2428, 805, 333, 2247, ...","[2, 4088, 321, 1533, 322, 2073, 401, 2013, 224..."
118000,"[2, 1235, 1117, 270, 8576, 265, 1592, 458, 461...","[2, 1954, 31979, 2735, 346, 4615, 9908, 224, 3..."
49446,"[2, 818, 775, 606, 398, 265, 1266, 224, 3, 0, ...","[2, 333, 1289, 627, 387, 410, 1390, 224, 3, 0,..."


In [2]:
import ast

df["English"] = df["English"].apply(ast.literal_eval)
df["Spanish"] = df["Spanish"].apply(ast.literal_eval)
df

,English,Spanish
47504,"[2, 265, 1225, 334, 3724, 332, 300, 1675, 224,...","[2, 323, 2116, 393, 15255, 355, 559, 224, 3, 0..."
73273,"[2, 395, 1089, 411, 300, 1647, 1191, 224, 3, 0...","[2, 1193, 1247, 331, 1316, 1313, 224, 3, 0, 0,..."
6048,"[2, 426, 449, 538, 224, 3, 0, 0, 0, 0, 0, 0, 0...","[2, 833, 444, 584, 224, 3, 0, 0, 0, 0, 0, 0, 0..."
21959,"[2, 479, 1873, 2127, 328, 224, 3, 0, 0, 0, 0, ...","[2, 801, 1880, 328, 7743, 224, 3, 0, 0, 0, 0, ..."
95763,"[2, 270, 384, 332, 300, 3142, 659, 300, 2110, ...","[2, 535, 621, 322, 331, 3381, 780, 331, 1934, ..."
...,...,...
16908,"[2, 426, 449, 395, 1320, 224, 3, 0, 0, 0, 0, 0...","[2, 444, 1368, 1756, 224, 3, 0, 0, 0, 0, 0, 0,..."
89015,"[2, 341, 304, 473, 300, 2428, 805, 333, 2247, ...","[2, 4088, 321, 1533, 322, 2073, 401, 2013, 224..."
118000,"[2, 1235, 1117, 270, 8576, 265, 1592, 458, 461...","[2, 1954, 31979, 2735, 346, 4615, 9908, 224, 3..."
49446,"[2, 818, 775, 606, 398, 265, 1266, 224, 3, 0, ...","[2, 333, 1289, 627, 387, 410, 1390, 224, 3, 0,..."


In [3]:
import torch
from torch.utils.data import DataLoader, TensorDataset

train = TensorDataset(
    torch.tensor(df["English"][:100000].tolist(), dtype=torch.long),
    torch.tensor(df["Spanish"][:100000].tolist(), dtype=torch.long),
)
test = TensorDataset(
    torch.tensor(df["English"][100000:].tolist(), dtype=torch.long),
    torch.tensor(df["Spanish"][100000:].tolist(), dtype=torch.long),
)
train_loader = DataLoader(train, batch_size=512, shuffle=True, drop_last=True)
test_loader = DataLoader(test, batch_size=512, shuffle=False, drop_last=True)

In [4]:
import json
with open(f"EnglishOrSpanish/vocab.json", "r") as f:
    vocab = json.load(f)

In [5]:
import torch
import torch.nn as nn


class Encoder(nn.Module):
    def __init__(
        self, vocab_size, embedding_dim, hidden_size, num_layers=2, dropout=0.3
    ):
        super(Encoder, self).__init__()
        self.num_layers = num_layers
        self.vocab_size = vocab_size

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
        )
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.LSTM):
            for name, param in module.named_parameters():
                if "weight" in name:
                    nn.init.xavier_uniform_(param)
                elif "bias" in name:
                    nn.init.zeros_(param)

    def forward(self, x):
        embedded = self.embedding(x)
        output, (hidden, cell) = self.lstm(embedded)
        return output, hidden, cell

In [6]:
import torch
import torch.nn as nn


class Decoder(nn.Module):
    def __init__(
        self, vocab_size, embedding_dim, hidden_size, num_layers=2, dropout=0.3
    ):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.vocab_size = vocab_size

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
        )
        self.fc = nn.Linear(hidden_size, vocab_size)
        self.apply(self._init_weights)
        self.dropout = nn.Dropout(dropout)

    def _init_weights(self, module):
        if isinstance(module, nn.LSTM):
            for name, param in module.named_parameters():
                if "weight" in name:
                    nn.init.xavier_uniform_(param)
                elif "bias" in name:
                    nn.init.zeros_(param)

    def forward(self, x, hidden, cell):
        x = self.embedding(x)
        x = x.unsqueeze(1)
        output, (hidden, cell) = self.lstm(x, (hidden, cell))
        output = self.dropout(output)
        output = self.fc(output)
        return output, hidden, cell

In [7]:
import torch.nn as nn
import torch
import random

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, vocab_size, teacher_forcing_ratio=0.5):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.vocab_size = vocab_size
        self.teacher_forcing_ratio = teacher_forcing_ratio

    def forward(self, src, trg):
        _, hidden, cell = self.encoder(src)
        outputs = torch.zeros(trg.shape[0], trg.shape[1], self.decoder.vocab_size).to("cuda")
        x = trg[:, 0]
        for t in range(1, trg.shape[1]):
            output, hidden, cell = self.decoder(x, hidden, cell)
            output = output.squeeze(1)
            outputs[:, t, :] = output
            use_teacher_forcing = random.random() < self.teacher_forcing_ratio
            x = trg[:, t] if use_teacher_forcing else output.argmax(dim=1)

        return outputs
    
    def predict(self, src):
        _, hidden, cell = self.encoder(src)
        outputs = torch.zeros(src.shape[0], src.shape[1], self.decoder.vocab_size).to("cuda")
        x = src[:, 0]
        for i in range(1, src.shape[1]):
            output, hidden, cell = self.decoder(x, hidden, cell)
            output = output.squeeze(1)
            outputs[:, i, :] = output
            x = output.argmax(dim=1)
        
        return outputs

In [9]:
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm


def train(
    model,
    train_loader,
    test_loader,
    optimizer,
    criterion,
    device,
    scheduler,
    num_epochs,
):
    writer = SummaryWriter()

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        model.encoder.train()
        model.decoder.train()
        epoch_loss = 0
        progress_bar = tqdm(train_loader, desc="Training", leave=False)

        for src, trg in progress_bar:
            src, trg = src.to(device), trg.to(device)
            optimizer.zero_grad()
            outputs = model(src, trg)
            loss = criterion(
                outputs[:, 1:].reshape(-1, model.vocab_size), trg[:, 1:].reshape(-1)
            )
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            progress_bar.set_postfix(loss=f"{loss.item():.4f}")

        train_loss = epoch_loss / len(train_loader)

        # ---------------- VALIDATION ---------------- #
        model.encoder.eval()
        model.decoder.eval()
        val_epoch_loss = 0
        progress_bar = tqdm(test_loader, desc="Validating", leave=False)

        with torch.no_grad():
            for src, trg in progress_bar:
                src, trg = src.to(device), trg.to(device)
                outputs = model(src, trg)
                loss = criterion(
                    outputs[:, 1:].reshape(-1, model.vocab_size),
                    trg[:, 1:].reshape(-1),
                )
                val_epoch_loss += loss.item()

                progress_bar.set_postfix(loss=f"{loss.item():.4f}")

        val_loss = val_epoch_loss / len(test_loader)
        scheduler.step(val_loss)

        writer.add_scalar("Loss/Train", train_loss, epoch)
        writer.add_scalar("Loss/Validation", val_loss, epoch)
        writer.add_scalar("Learning Rate", scheduler.get_last_lr()[0], epoch)

        print(
            f"Train loss : {train_loss}, Val_loss : {val_loss}, lr: {scheduler.get_last_lr()[0]}"
        )

    writer.close()
    return train_loss, val_loss

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vocab_size = len(vocab)
embedding_dim = 512
hidden_size = 64
encoder = Encoder(vocab_size, embedding_dim, hidden_size).to(device)
decoder = Decoder(vocab_size, embedding_dim, hidden_size).to(device)
seq2seq = Seq2Seq(encoder, decoder, vocab_size).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(seq2seq.parameters(), lr=0.005, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=2
)

In [11]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

train(
    seq2seq,
    train_loader,
    test_loader,
    optimizer,
    criterion,
    device,
    scheduler,
    30,
)


Epoch 1/30


Train loss : 6.687192748143123, Val_loss : 5.654639618737357, lr: 0.001

Epoch 2/30


Train loss : 5.6322522994799495, Val_loss : 5.445596354348319, lr: 0.001

Epoch 3/30


Train loss : 5.444505283160088, Val_loss : 5.27954580954143, lr: 0.001

Epoch 4/30


Train loss : 5.288971583048503, Val_loss : 5.134128902639661, lr: 0.001

Epoch 5/30


...

Train loss : 3.623416779591487, Val_loss : 3.584328361919948, lr: 0.001

Epoch 28/30


Train loss : 3.6013498257368037, Val_loss : 3.5853850288050517, lr: 0.001

Epoch 29/30


Train loss : 3.5387104499034394, Val_loss : 3.490789668900626, lr: 0.001

Epoch 30/30


Train loss : 3.518312445665017, Val_loss : 3.470277441399438, lr: 0.001


(3.518312445665017, 3.470277441399438)

In [12]:
torch.save(seq2seq.state_dict(), "seq2seq-noattention.pth")

# Evaluation

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vocab_size = len(vocab)
embedding_dim = 512
hidden_size = 64
encoder = Encoder(vocab_size, embedding_dim, hidden_size).to(device)
decoder = Decoder(vocab_size, embedding_dim, hidden_size).to(device)
seq2seq = Seq2Seq(encoder, decoder, vocab_size).to(device)
seq2seq.load_state_dict(torch.load("seq2seq-noattention.pth", weights_only=True))

<All keys matched successfully>

In [9]:
seq2seq.eval()
# with torch.no_grad():
a, b = next(iter(test_loader))
a = a.to(device)
b = b.to(device)
with torch.no_grad():
    x = seq2seq.predict(a)

x = x.argmax(dim=2)
x.shape

torch.Size([512, 18])

In [10]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer(
    f"EnglishOrSpanish/vocab.json", f"EnglishOrSpanish/merges.txt"
)

In [11]:
x = x.cpu().numpy()
b = b.cpu().numpy()
a = a.cpu().numpy()
preds = []
targets = []
originals = []
for i in range(x.shape[0]):
    preds.append(tokenizer.decode(x[i]))
    targets.append(tokenizer.decode(b[i]))
    originals.append(tokenizer.decode(a[i]))


In [12]:
import pandas as pd

df = pd.DataFrame(
    {"Original": originals[:10], "Prediction": preds[:10], "Target": targets[:10]}
)
df.to_csv("no-attention.csv", index=False)
df

,Original,Prediction,Target
0,<SOS> i am aware of the fact <EOS><PAD><PAD><P...,<PAD> estoy estoy de <EOS><EOS><EOS><EOS><EOS...,<SOS> estoy informado del hecho <EOS><PAD><PAD...
1,<SOS> i hope everyone agrees <EOS><PAD><PAD><P...,<PAD> me que que de de de <EOS><EOS><EOS><EOS...,<SOS> confío en que toda la gente esté de acue...
2,<SOS> can you lend me a dollar <EOS><PAD><PAD>...,<PAD> puedes que un un <EOS><EOS><EOS><EOS><E...,<SOS> puedes prestarme un dólar <EOS><PAD><PAD...
3,<SOS> you may invite whoever you like <EOS><PA...,<PAD> puedes que que que que <EOS><EOS><EOS><...,<SOS> puedes invitar a quien quieras <EOS><PAD...
4,<SOS> it is just your imagination <EOS><PAD><P...,<PAD> es es un <EOS><EOS><EOS><EOS><EOS><EOS>...,<SOS> es sólo tu imaginación <EOS><PAD><PAD><P...
5,<SOS> dont worry tom wont let us down <EOS><PA...,<PAD> no no que tom no no a a <EOS><EOS><EOS><...,<SOS> no se preocupe tom no nos va a decepcion...
6,<SOS> the problem is that some of our bills ha...,<PAD> el libro que es que no no no de de de de...,<SOS> el problema es que algunas de nuestras c...
7,<SOS> we kissed in the dark <EOS><PAD><PAD><PA...,<PAD> nos a la la <EOS><EOS><EOS><EOS><EOS><EO...,<SOS> nos besamos en la oscuridad <EOS><PAD><P...
8,<SOS> you can tell tom yourself <EOS><PAD><PAD...,<PAD> te que que que tom <EOS><EOS><EOS><EOS><...,<SOS> puedes decírselo a tom tú mismo <EOS><PA...
9,<SOS> she could not cope with anxiety <EOS><PA...,<PAD> ella no le que a la <EOS><EOS><EOS><EOS...,<SOS> ella no pudo hacer frente a la ansiedad ...


In [ ]:
import sacrebleu


def calculate_bleu_score(preds, targets):
    bleu = sacrebleu.corpus_bleu(preds, [targets], force=True)
    return bleu.score

calculate_bleu_score(preds, targets)

15.419322188276391

In [12]:
import rouge

rouge = rouge.Rouge()
scores = rouge.get_scores(preds, targets, avg=True)
scores

{'rouge-1': {'r': 0.1662537654466656,
  'p': 0.24795231894841332,
  'f': 0.1958045229363409},
 'rouge-2': {'r': 0.03523896351044787,
  'p': 0.04266514193272004,
  'f': 0.03831203220767204},
 'rouge-l': {'r': 0.16118833662635393,
  'p': 0.23948955233134994,
  'f': 0.18959254059853015}}

In [15]:
def ter(preds, targets):
    return sacrebleu.corpus_ter(preds, [targets]).score

ter(preds, targets)

86.74728529121421